In [48]:
import pandas as pd
import numpy as np
from functools import reduce
import os
import pickle

DATASET = 'TN'
# TN QB NERD

MODEL = 'NARM+'
# GRU4Rec, NARM, BERT4Rec, GRU4Rec+, NARM+, BERT4Rec+, where + means the enhanced model with HealthRec framework

# with open(os.path.join("..", "parsed_data", "NARM", DATASET, "H_label.pkl"), "rb") as pf:
#     ground = pickle.load(pf)

with open(os.path.join(".", "datasets", DATASET, "H_label.pkl"), "rb") as pf:
    ground = pickle.load(pf)

# position 0 is the padding, do not use!
# print(len(ground))

with open(f"./datasets/{DATASET}/prediction_health_{DATASET}_{MODEL}.txt", "rb") as pf:
# with open(f"../parsed_data/LESSRDisen/{DATASET}/prediction_MSGIFSR.txt", "rb") as pf:
    prediction = pickle.load(pf)

if len(prediction) == 3:
    # the old logit output format used in NARM
    prediction = prediction[2]
elif len(prediction) == 2:
    prediction = prediction[1]
prediction = np.array(prediction)
prediction = prediction.reshape(-1, 20)


numID_health = {i:h for i, h in enumerate(ground)}

top20 = prediction
health_top20 = []
for toplist in top20:
    health_top20.append(list(map(lambda x: numID_health[x], toplist)))

health_top20 = np.array(health_top20)
h1 = health_top20[:, 0]
h5 = health_top20[:, 0:5].sum(axis=1)/5
h10 = health_top20[:, 0:10].sum(axis=1)/10
h20 = health_top20[:, 0:20].sum(axis=1)/20

print("Dataset: ",DATASET," Model: ",MODEL)

print("\033[32mHP@1\tHP@5\tHP@10\tHP@20\033[0m")
print("%.2f\t%.2f\t%.2f\t%.2f" % (h1.mean()*100, h5.mean()*100, h10.mean()*100, h20.mean()*100))

rank_mat = np.arange(1,21)
rr_mat = 1/rank_mat
rr_score = rr_mat * health_top20
rr1 = rr_score[:, 0]
rr5 = rr_score[:, 0:5].sum(axis=1)/rr_mat[0:5].sum()
rr10 = rr_score[:, 0:10].sum(axis=1)/rr_mat[0:10].sum()
rr20 = rr_score[:, 0:20].sum(axis=1)/rr_mat[0:20].sum()

print("\033[32mHO@1\tHO@5\tHO@10\tHO@20\033[0m")
print("%.2f\t%.2f\t%.2f\t%.2f" % (rr1.mean()*100, rr5.mean()*100, rr10.mean()*100, rr20.mean()*100))

Dataset:  NERD  Model:  NARM+
HP@1	HP@5	HP@10	HP@20
59.29	56.73	55.88	54.64
HO@1	HO@5	HO@10	HO@20
59.29	57.54	57.01	56.37
